# ARTIFICIAL NEURAL NETWORKS

## Task 1: Data Exploration and Preprocessing
### Import Libraries

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam


from scikeras.wrappers import KerasClassifier

### Load Dataset

In [27]:
df = pd.read_csv("sonardataset.csv", header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10,...,x_52,x_53,x_54,x_55,x_56,x_57,x_58,x_59,x_60,Y
1,0.02,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.018,0.0084,0.009,0.0032,R
2,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.014,0.0049,0.0052,0.0044,R
3,0.0262,0.0582,0.1099,0.1083,0.0974,0.228,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.018,0.0244,0.0316,0.0164,0.0095,0.0078,R
4,0.01,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.015,0.0085,0.0073,0.005,0.0044,0.004,0.0117,R


### Dataset Overview

In [29]:
print("Dataset Shape:", df.shape)
print("Class Distribution:\n")
print(df.iloc[:, -1].value_counts())

Dataset Shape: (209, 61)
Class Distribution:

60
M    111
R     97
Y      1
Name: count, dtype: int64


### Separate Features and Target

In [32]:
X = df.iloc[:, :-1] # First 60 columns
y = df.iloc[:, -1] # Target column

### Convert Features to Numeric

In [35]:
X = X.apply(pd.to_numeric, errors="coerce")
X = X.fillna(0)

### Encode Target Labels

In [38]:
le = LabelEncoder()
y = le.fit_transform(y) # M=1, R=0

### Feature Scaling

In [41]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

### Train-Test Split

In [44]:
X_train, X_test, y_train, y_test = train_test_split(
X_scaled, y, test_size=0.2, random_state=42
)

## Task 2: ANN Model Implementation
### Baseline ANN Model

In [49]:
def create_baseline_model():
    model = Sequential([
        Dense(32, activation='relu', input_shape=(60,)),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

### Train Baseline Model

In [53]:
baseline_model = create_baseline_model()
baseline_model.fit(
X_train, y_train,
epochs=50,
batch_size=16,
validation_split=0.1,
verbose=0
)

C:\Users\shree\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Evaluate Baseline Model

In [56]:
y_pred = (baseline_model.predict(X_test) > 0.5).astype(int)


print("Baseline Model Performance")
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Baseline Model Performance
Accuracy: 0.7857142857142857
              precision    recall  f1-score   support

           0       0.81      0.84      0.82        25
           1       0.75      0.71      0.73        17

    accuracy                           0.79        42
   macro avg       0.78      0.77      0.78        42
weighted avg       0.78      0.79      0.78        42



## Task 3: Hyperparameter Tuning (SciKeras)
### Model Builder Function

In [59]:
def build_model(neurons=32, learning_rate=0.001):
    model = Sequential()
    model.add(Dense(neurons, activation='relu', input_shape=(60,)))
    model.add(Dense(neurons//2, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))


    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

### Grid Search Setup

In [62]:
keras_clf = KerasClassifier(
model=build_model,
epochs=50,
batch_size=16,
verbose=0
)


param_grid = {
"model__neurons": [16, 32, 64],
"model__learning_rate": [0.01, 0.001, 0.0001]
}


grid = GridSearchCV(
estimator=keras_clf,
param_grid=param_grid,
cv=3,
scoring='accuracy'
)


grid_result = grid.fit(X_train, y_train)

C:\Users\shree\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(
C:\Users\shree\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\shree\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


C:\Users\shree\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\shree\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\shree\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwa

### Best Hyperparameters

In [64]:
print("Best Cross-Validation Accuracy:", grid_result.best_score_)
print("Best Parameters:", grid_result.best_params_)

Best Cross-Validation Accuracy: 0.6161255411255412
Best Parameters: {'model__learning_rate': 0.01, 'model__neurons': 32}


## Task 4: Final Evaluation (Tuned Model)

In [68]:
best_model = grid_result.best_estimator_
y_pred_best = best_model.predict(X_test)


print("Tuned Model Performance")
print("Accuracy:", accuracy_score(y_test, y_pred_best))
print(classification_report(y_test, y_pred_best))

Tuned Model Performance
Accuracy: 0.5952380952380952
              precision    recall  f1-score   support

           0       0.60      1.00      0.75        25
           1       0.00      0.00      0.00        17

    accuracy                           0.60        42
   macro avg       0.30      0.50      0.37        42
weighted avg       0.35      0.60      0.44        42



C:\Users\shree\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\shree\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\shree\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Conclusion

ANN successfully classified sonar signals into Mine and Rock categories

Proper preprocessing was critical to avoid scaling errors

Hyperparameter tuning significantly improved performance

ANN is effective for sonar-based object classification